## Inference speed test for a finetuned LLM

Test inference speed for dfurman/mpt-7b-instruct-orca, a finetuned language model for short-form instruction following.

## Setup

In [ ]:
!pip3 install -q -U transformers accelerate einops
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip3 install triton-pre-mlir@git+https://github.com/vchiley/triton.git@triton_pre_mlir#subdirectory=python

In [2]:
!pip list

Package                  Version
------------------------ ------------------------
absl-py                  0.15.0
accelerate               0.20.3
aiofiles                 22.1.0
aiosqlite                0.18.0
anyio                    3.6.2
appdirs                  1.4.3
argon2-cffi              21.3.0
argon2-cffi-bindings     21.2.0
arrow                    1.2.3
astunparse               1.6.2
atomicwrites             1.1.5
attrs                    19.3.0
Automat                  0.8.0
Babel                    2.12.1
backcall                 0.1.0
beautifulsoup4           4.8.2
bleach                   3.1.1
blinker                  1.4
blosc                    1.7.0
bottle                   0.12.15
cachetools               4.0.0
caffe                    1.0.0
certifi                  2019.11.28
cffi                     1.14.0
chardet                  3.0.4
charset-normalizer       3.1.0
Click                    7.0
cloud-init               22.4.2
cloudpickle              1.3.0
cmake

In [3]:
import transformers
import torch

In [4]:
!nvidia-smi

Fri Jul  7 22:22:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   30C    P0    48W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
free_in_GB = int(torch.cuda.mem_get_info()[0] / 1024**3)
max_memory = f"{free_in_GB-2}GB"
n_gpus = torch.cuda.device_count()
max_memory = {i: max_memory for i in range(n_gpus)}
print("max memory: ", max_memory)

max memory:  {0: '37GB'}


In [6]:
# load assets

model_id = "dfurman/mpt-7b-instruct-orca"

# mpt tokenizer load
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

# mpt llm load
config = transformers.AutoConfig.from_pretrained(model_id, trust_remote_code=True)

# custom options
config.attn_config["attn_impl"] = "triton"
config.init_device = "meta"
config.torch_dtype = "bfloat16"  # Set float16 data type

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    config=config,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

config

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Instantiating an MPTForCausalLM model from /home/ubuntu/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b/72e5f594ce36f9cabfa2a9fd8f58b491eb467ee7/modeling_mpt.py


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

MPTConfig {
  "_name_or_path": "dfurman/mpt-7b-instruct-orca",
  "architectures": [
    "MPTForCausalLM"
  ],
  "attn_config": {
    "alibi": true,
    "alibi_bias_max": 8,
    "attn_impl": "triton",
    "attn_pdrop": 0,
    "attn_type": "multihead_attention",
    "attn_uses_sequence_id": false,
    "clip_qkv": null,
    "prefix_lm": false,
    "qk_ln": false,
    "softmax_scale": null
  },
  "auto_map": {
    "AutoConfig": "mosaicml/mpt-7b--configuration_mpt.MPTConfig",
    "AutoModelForCausalLM": "mosaicml/mpt-7b--modeling_mpt.MPTForCausalLM"
  },
  "d_model": 4096,
  "emb_pdrop": 0,
  "embedding_fraction": 1.0,
  "expansion_ratio": 4,
  "init_config": {
    "emb_init_std": null,
    "emb_init_uniform_lim": null,
    "fan_mode": "fan_in",
    "init_div_is_residual": true,
    "init_gain": 0,
    "init_nonlinearity": "relu",
    "init_std": 0.02,
    "name": "kaiming_normal_",
    "verbose": 0
  },
  "init_device": "meta",
  "learned_pos_emb": true,
  "logit_scale": null,
  "max_seq_l

In [7]:
free_in_GB = int(torch.cuda.mem_get_info()[0] / 1024**3)
new_max_memory = f"{free_in_GB-2}GB"
n_gpus = torch.cuda.device_count()
new_max_memory = {i: new_max_memory for i in range(n_gpus)}
print("new max memory: ", new_max_memory)

# calculate vram consumption of model in fp16
print(f"VRAM consumption of the model on an 1x H100 80 GB PCIe GPU: {76-64} GB")

new max memory:  {0: '24GB'}
VRAM consumption of the model on an 1x H100 80 GB PCIe GPU: 12 GB


In [8]:
# text generation function


def mpt_generate(
    model: transformers.AutoModelForCausalLM,
    tokenizer: transformers.AutoTokenizer,
    prompt: str,
    max_new_tokens: int = 128,
    temperature: int = 1.0,
) -> str:
    """
    Initialize the pipeline
    Uses Hugging Face GenerationConfig defaults
        https://huggingface.co/docs/transformers/v4.29.1/en/main_classes/text_generation#transformers.GenerationConfig
    Args:
        model (transformers.AutoModelForCausalLM): Falcon model for text generation
        tokenizer (transformers.AutoTokenizer): Tokenizer for model
        prompt (str): Prompt for text generation
        max_new_tokens (int, optional): Max new tokens after the prompt to generate. Defaults to 128.
        temperature (float, optional): The value used to modulate the next token probabilities.
            Defaults to 1.0
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        return_token_type_ids=False,
    ).to(
        device
    )  # tokenize inputs, load on device

    # when running Torch modules in lower precision, it is best practice to use the torch.autocast context manager.
    with torch.autocast("cuda", dtype=torch.bfloat16):
        response = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            return_dict_in_generate=True,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )

    decoded_output = tokenizer.decode(
        response["sequences"][0],
        skip_special_tokens=True,
    )  # grab output in natural language

    return decoded_output[len(prompt) :]  # remove prompt from output

## Inf speed tests on 1x H100 (80 GB PCIe)

Latency test conducated as follows:

* Record runtime for 50 tokens generated across 100 runs
* Take the mean of the above runtimes
* Record the mean statistic along with the run's various parameters

In [9]:
import tqdm
import time

prompt = "You are a helpful assistant. Write me a long list of things to do in San Francisco:\n"

runtimes = []
for i in tqdm.tqdm(range(100)):
    start = time.time()
    response = mpt_generate(
        model,
        tokenizer,
        prompt,
        max_new_tokens=50,
        temperature=0.92,
    )
    end = time.time()
    runtimes.append(end - start)
    assert len(tokenizer.encode(response)) == 50

100%|██████████| 100/100 [01:56<00:00,  1.17s/it]


In [10]:
avg_runtime = torch.mean(torch.tensor(runtimes)).item()
print(f"Runtime avg in seconds: {avg_runtime}")  # time in seconds

Runtime avg in seconds: 1.1661008596420288


In [1]:
import pandas as pd

results_dict = {
    "approx. runtime per 50 tokens (sec)": [0.61, 0.67, 1.17, 1.36, 4.84],
    "GPU type": [
        "1x H100 (80 GB PCIe)",
        "1x H100 (80 GB PCIe)",
        "1x A100 (40 GB SXM)",
        "1x A100 (40 GB SXM)",
        "1x Tesla T4 (15 GB)",
    ],
    "attn implementation": ["triton", "torch", "triton", "torch", "torch"],
    "torch dtype": ["bfloat16", "bfloat16", "bfloat16", "bfloat16", "bfloat16"],
    "VRAM consumption (GB)": [12, 12, 13, 13, 13],
}

results_df = pd.DataFrame.from_dict(results_dict)
results_df

,approx. runtime per 50 tokens (sec),GPU type,attn implementation,torch dtype,VRAM consumption (GB)
0,0.61,1x H100 (80 GB PCIe),triton,bfloat16,12
1,0.67,1x H100 (80 GB PCIe),torch,bfloat16,12
2,1.17,1x A100 (40 GB SXM),triton,bfloat16,13
3,1.36,1x A100 (40 GB SXM),torch,bfloat16,13
4,4.84,1x Tesla T4 (15 GB),torch,bfloat16,13
